In [3]:
import pandas as pd
import numpy as np

In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error


In [5]:
df = pd.read_parquet('./yellow_tripdata_2023-01.parquet')

In [6]:
len(df.columns)

19

**✅ The number of columns in the dataset is 19.**

In [7]:
#df

In [8]:
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

df = df[df['duration'] > 0]

std_duration = df['duration'].std()

print(f"Standard Deviation of Trip Durations: {round(std_duration, 3)}")

Standard Deviation of Trip Durations: 42.601


**✅ Standard Deviation of Trip Durations: 42.601.**

In [9]:
initial_count = len(df)

df_filtered = df[(df['duration'] >= 1) & (df['duration'] <= 60)]

filtered_count = len(df_filtered)

fraction = filtered_count / initial_count

print(f"Fraction of records left: {round(fraction * 100, 2)}%")

Fraction of records left: 98.16%


**✅ Fraction of records left: 98.16%.**

In [10]:
df_enc = df_filtered[['PULocationID', 'DOLocationID']].copy()

df_enc['PULocationID'] = df_enc['PULocationID'].astype(str)
df_enc['DOLocationID'] = df_enc['DOLocationID'].astype(str)

dicts = df_enc.to_dict(orient='records')



In [11]:
from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer()
X = dv.fit_transform(dicts)

In [12]:
X.shape

(3009173, 515)

**✅ number of columns 515.**

In [13]:
y = df_filtered['duration']

model = LinearRegression()
model.fit(X, y)

y_pred = model.predict(X)

rmse = mean_squared_error(y, y_pred, squared=False)
print(f"RMSE: {round(rmse, 2)}")

RMSE: 7.65


**✅ RMSE is 7.65.**

In [14]:
df_val = pd.read_parquet('yellow_tripdata_2023-02.parquet')

df_val['tpep_pickup_datetime'] = pd.to_datetime(df_val['tpep_pickup_datetime'])
df_val['tpep_dropoff_datetime'] = pd.to_datetime(df_val['tpep_dropoff_datetime'])

df_val['duration'] = (df_val['tpep_dropoff_datetime'] - df_val['tpep_pickup_datetime']).dt.total_seconds() / 60

df_val = df_val[(df_val['duration'] >= 1) & (df_val['duration'] <= 60)]

In [15]:
df_val_enc = df_val[['PULocationID', 'DOLocationID']].copy()
df_val_enc['PULocationID'] = df_val_enc['PULocationID'].astype(str)
df_val_enc['DOLocationID'] = df_val_enc['DOLocationID'].astype(str)

dicts_val = df_val_enc.to_dict(orient='records')

In [16]:
X_val = dv.transform(dicts_val)

y_val = df_val['duration']
y_pred = model.predict(X_val)

In [17]:
rmse_val = mean_squared_error(y_val, y_pred, squared=False)

print(f"Validation RMSE: {round(rmse_val, 2)}")

Validation RMSE: 7.81


**✅ Validation RMSE: 7.81.**